### A notebook environment is the perfect place to test & explore

* I started off with pytesseract, but after some research found PaddleOCR to have a higher accuracy & be much easier to implement.
* PaddleOCR on its own is quite good - but not immune. I'll support it with the proper preprocessing:

In [6]:
import os
from PIL import Image
from paddleocr import PaddleOCR
import os
import cv2
import numpy as np
from PIL import Image
import pytesseract
from itertools import product
import os, cv2
from PIL import Image
input_folder = 'images/lab_tests_origin'
output_folder = 'images/lab_tests_png'
ocr = PaddleOCR(use_angle_cls=True, lang='en')
# This will take some time to fully install all models needed for PaddleOCR.

C:\Users\User\AppData\Local\Temp\ipykernel_4532\1275244073.py:14: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=True, lang='en')
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\User\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\User\.paddlex\official_models\UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\User\.paddlex\official_models\PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please dele

In [ ]:
allowed_ext = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".tif")

In [17]:

import cv2
import numpy as np
import cv2
import numpy as np

def preprocess_for_ocr(image_path):
    # Read image in BGR
    img = cv2.imread(image_path)

    # Convert to grayscale for preprocessing steps
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Increase contrast using CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    contrast = clahe.apply(gray)

    # Adaptive thresholding
    thresh = cv2.adaptiveThreshold(
        contrast,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        31,
        10
    )

    # Noise removal
    kernel = np.ones((2, 2), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Convert back to 3-channel BGR for PaddleOCR
    final_img = cv2.cvtColor(cleaned, cv2.COLOR_GRAY2BGR)

    return final_img


In [ ]:
filepath = 'C:/Users/User/Documents/LabIQ/images/testing/2.png'
image = cv2.imread(filepath)
# image = preprocess_for_ocr(filepath) 
prediction = ocr.predict(image)
for text, score in zip(prediction[0]['rec_texts'], prediction[0]['rec_scores']):
    print(f"{text} ({score:.2f})")

DRLOGY PATHOLOGY LAB (1.00)
0123456789 | 0912345678 (0.99)
Accurate (1.00)
Caring | Instant (0.95)
drlogypathlab@drlogy.com (1.00)
www.drlogy.com (1.00)
Yash M. Patel (1.00)
Sample Collected At: (0.99)
Age : 21 Years (0.93)
125, Shivam Bungalow, S G Road, (0.99)
0 35545 62336 78 1 (0.97)
Sex : Male (0.97)
Registered on: 02:31 PM 02 Dec, 2X (0.98)
Mumbai (1.00)
Collected on: 03:11 PM 02 Dec, 2X (0.99)
PID : 555 (0.97)
Ref. By:Dr. Hiren Shah (0.98)
Reported on: 04:35 PM 02 Dec, 2X (0.98)
Complete Blood Count (CBC) with ESR (0.97)
Investigation (1.00)
Result (1.00)
Reference Value (0.97)
Unit (1.00)
Primary Sample Type : (0.99)
Blood (1.00)
HEMOGLOBIN (1.00)
Hemoglobin (Hb) (1.00)
12.5 (1.00)
Low 13.0 -17.0 (0.97)
g/dL (1.00)
RBC COUNT (0.96)
Total RBC count (1.00)
5.2 (1.00)
4.5 - 5.5 (0.99)
mill/cumm (1.00)
BLOOD INDICES (0.99)
Packed Cell Volume (PCV) (1.00)
57.5 (1.00)
High (0.91)
40 - 50 (0.94)
% (1.00)
Mean Corpuscular Volume (MCV) (0.97)
87.75 (1.00)
83 - 101 (0.93)
fL (1.00)
Calcu

#### Image 5 did great with preprocessing; but this **does not mean all images will**.

In [ ]:
filepath = 'C:/Users/User/Documents/LabIQ/images/testing/1.png'
image = cv2.imread(filepath)
image = preprocess_for_ocr(filepath) 
prediction = ocr.predict(image)
for text, score in zip(prediction[0]['rec_texts'], prediction[0]['rec_scores']):
    print(f"{text} ({score:.2f})")
print()
print("="*50)
print()

Conclusion:
* I will try both aggressive preprocessing and none on the input image, dictating the best method according to its CER.
* To truly evaluate my OCR results I'll find the CER (character error rate) between the predictions & ground truths. ( You'll notice in the ground_truths I kept some mistakes in the parts of the blood tests that I won't extract entities from such as name and sex, as penalizing the model on text that won't be used down the line doesn't reflect our objective. )

In [ ]:
filepath = 'C:/Users/User/Documents/LabIQ/images/testing/1.png'
image = cv2.imread(filepath) 
prediction = ocr.predict(image)
for text, score in zip(prediction[0]['rec_texts'], prediction[0]['rec_scores']):
    print(f"{text} ({score:.2f})")
print()
print("="*50)
print()

Now we will use regular expressions to extract entity:value pairs, then transform our data into a pandas DataFrame.
* I chose regex functions for this stage because it is unlikely that a pre-trained NER model exists to specifically categorize blood test entities (Analyte, Unit, Reference Range, etc.). If I find any NER model for this specific purpose I'll be sure to make the necessary changes.

In [ ]:
import re

def classify_token(text):
    text = text.strip()

    if not text or len(text) <= 1:
        return "NOISE"

    # Reference ranges: 11.5 - 14.5, 4–11, 13 to 17
    if re.search(r"\d+(\.\d+)?\s*[-–to]+\s*\d+(\.\d+)?", text, re.I):
        return "RANGE"

    # Inequality ranges: <2, <= 15, Up to 15
    if re.search(r"^(<|>|≤|≥)\s*\d+(\.\d+)?", text):
        return "RANGE"
    if re.search(r"up to\s*\d+(\.\d+)?", text, re.I):
        return "RANGE"

    # Numeric value (including flags like *, L, H)
    if re.fullmatch(r"\d+(\.\d+)?\s*[*HL]?", text):
        return "VALUE"

    # Units
    if re.search(r"(g/dl|mg/dl|mm/hr|%|fl|pg|u/l|x10\^?\d+|/uL|/cumm)", text, re.I):
        return "UNIT"

    # Test names (CBC, Hemoglobin, WBC, Platelet Count, etc.)
    if re.search(r"(hemoglobin|wbc|rbc|platelet|hematocrit|mcv|mchc|rdw|neutrophil|lymphocyte|cbc)", text, re.I):
        return "ANALYTE"

    # Metadata
    if re.search(r"(age|gender|male|female|date|doctor|patient|report|lab)", text, re.I):
        return "META"

    return "TEXT"


In [ ]:
results = dict()

for text, score in zip(prediction[0]['rec_texts'], prediction[0]['rec_scores']):
        if score> 0.85:
                results[text] = classify_token(text)
        else:
                continue
print(results)

{'DRLOGY PATHOLOGY LAB': 'META', '0123456789 | 0912345678': 'TEXT', 'Accurate': 'TEXT', '|Caring |Instant': 'TEXT', 'drlogypathlab@drlogy.com': 'META', 'www.drlogy.com': 'TEXT', 'Yash M. Patel': 'TEXT', 'Sample Collected At:': 'TEXT', 'Age : 21 Years': 'META', '125, Shivam Bungalow, S G Road,': 'TEXT', '0 35545 62336 78 1': 'TEXT', 'Sex : Male': 'META', 'Registered on: 02:31 PM 02 Dec, 2X': 'TEXT', 'Mumbai': 'TEXT', 'Collected on: 03:11 PM 02 Dec, 2X': 'TEXT', 'PID : 555': 'TEXT', 'Ref. By: Dr. Hiren Shah': 'TEXT', 'Reported on: 04:35 PM 02 Dec, 2X': 'META', 'Complete Blood Count (CBC) with ESR': 'ANALYTE', 'Investigation': 'TEXT', 'Result': 'TEXT', 'Reference Value': 'TEXT', 'Unit': 'TEXT', 'Primary Sample Type :': 'TEXT', 'Blood': 'TEXT', 'HEMOGLOBIN': 'ANALYTE', 'Hemoglobin (Hb)': 'ANALYTE', '12.5': 'VALUE', 'Low 13.0 - 17.0': 'RANGE', 'g/dL': 'UNIT', 'RBC COUNT': 'ANALYTE', 'Total RBC count': 'ANALYTE', '5.2': 'VALUE', '4.5 - 5.5': 'RANGE', 'mill/cumm': 'UNIT', 'BLOOD INDICES': 'TE

In [ ]:
def assemble_lab_rows(tokens):
    rows = []
    current = {}

    for token, label in tokens:
        if label == "ANALYTE":
            if current:
                rows.append(current)
            current = {"Analyte": token}

        elif label == "VALUE":
            current["value"] = token

        elif label == "UNIT":
            current["unit"] = token

        elif label == "RANGE":
            current["range"] = token

    if current:
        rows.append(current)

    return rows
import pandas as pd
df = pd.DataFrame()

for item in results:
    df[''] = assemble_lab_rows

In [ ]:
def is_abnormal(value, ref_range):
    try:
        low, high = map(float, re.split(r"[-–to]+", ref_range))
        v = float(re.sub(r"[*HL]", "", value))
        if v < low:
            return "LOW"
        if v > high:
            return "HIGH"
        return "NORMAL"
    except:
        return "UNKNOWN"
